In [ ]:
import numpy as np
import torch
file_path = '../sample_file/q_map_0.txt'
matrix = np.loadtxt(file_path)
matrix = matrix.reshape(1,1,128,128)
matrix = torch.tensor(matrix)
matrix

In [ ]:
def argmax2d(tensor):
  n = tensor.size(0)
  d = tensor.size(2)
  m = tensor.view(n, -1).argmax(1)
  return torch.cat(((m / d).view(-1, 1), (m % d).view(-1, 1)), dim=1)

argmax2d(matrix)[0]

In [ ]:
copy_matrix = matrix.clone()
for i in range(100):
    max_index = argmax2d(copy_matrix)[0].long()
    print(max_index, copy_matrix[0][0][max_index[0]][max_index[1]])
    copy_matrix[0][0][max_index[0]][max_index[1]] = 0

In [ ]:
import numpy as np
def argmax2d(tensor):
    n = tensor.size(0)
    d = tensor.size(2)
    m = tensor.view(n, -1).argmax(1)
    return torch.cat(((m / d).view(-1, 1), (m % d).view(-1, 1)), dim=1)

def set_center_region(submatrix, n, value):
    # 获取中心区域的边界
    min_row = submatrix.shape[0] // 2 - n // 2
    max_row = min_row + n
    min_col = submatrix.shape[1] // 2 - n // 2
    max_col = min_col + n

    # 将中心区域的值设置为指定值
    submatrix[min_row:max_row, min_col:max_col] = value
    return submatrix


def find_second_max(matrix, center, neighborhood_size=20, min_distance=10):

    a, b = center

    # 定义邻域的边界
    min_row = max(0, a - neighborhood_size // 2)
    max_row = min(matrix.shape[0], a + neighborhood_size // 2 + 1)
    min_col = max(0, b - neighborhood_size // 2)
    max_col = min(matrix.shape[1], b + neighborhood_size // 2 + 1)

    # 在邻域内找到次最大值的坐标
    submatrix = matrix[min_row:max_row, min_col:max_col]
    submatrix = set_center_region(submatrix, min_distance, 0)
    flattened_indices = np.argsort(submatrix, axis=None)  # 将邻域展平后的索引
    second_max_index = np.unravel_index(flattened_indices[-2], submatrix.shape)

    # 转换为全局坐标
    second_max_coord = (a - neighborhood_size // 2 + second_max_index[0],
                        b - neighborhood_size // 2 + second_max_index[1])

    return second_max_coord

file_path = '../sample_file/q_map_1.txt'
matrix = np.loadtxt(file_path)
matrix = torch.tensor(matrix)

# 找到最大值的坐标
max_coords = argmax2d(matrix.reshape(1,1,128,128)).long()[0]

# 找到次最大值的坐标
second_max_coords = find_second_max(matrix, max_coords)

print("最大值坐标:", max_coords)
print("次最大值坐标:", second_max_coords)


In [ ]:
import re
import matplotlib.pyplot as plt

# 定义一个字典来存储每个 action 的累计 reward 和计数器
action_rewards = {}

# 打开文件并逐行读取
with open("../temp/object_attacked_position.txt") as file:
    for line in file:
        # 解析每一行，提取出 action 和对应的 reward
        values = [float(match) for match in re.findall(r'[-+]?\d*\.\d+(?:[eE][-+]?\d+)?|\d+', line)]
        action = tuple(values[7:11])  # 提取 action，这里假设 action 是一个由四个元素组成的元组
        reward = values[-1]  # 提取 reward

        # 更新字典中对应 action 的累计 reward 和计数器
        if action in action_rewards:
            action_rewards[action]['total_reward'] += reward
            action_rewards[action]['count'] += 1
        else:
            action_rewards[action] = {'total_reward': reward, 'count': 1}

# 计算每个 action 的平均 reward
average_rewards = {}
for action, reward_info in action_rewards.items():
    average_reward = reward_info['total_reward'] / reward_info['count']
    average_rewards[action] = average_reward

# 提取每个动作的x和y坐标
x_coords = [action[0] for action in average_rewards.keys()]
y_coords = [action[1] for action in average_rewards.keys()]

# 画图
plt.figure(figsize=(30,30))
plt.scatter(x_coords, y_coords, s=30, c=list(average_rewards.values()), cmap='viridis')
plt.colorbar(label='Average Reward')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Average Reward for Each Action')
plt.grid(True)
plt.show()


In [ ]:
import re
import matplotlib.pyplot as plt
import numpy as np

# 定义一个字典来存储每个 action 的累计 reward 和计数器
action_rewards = {}

# 打开文件并逐行读取
with open("../temp/object_attacked_position.txt") as file:
    for line in file:
        # 解析每一行，提取出 action 和对应的 reward
        values = [float(match) for match in re.findall(r'[-+]?\d*\.\d+(?:[eE][-+]?\d+)?|\d+', line)]
        action = tuple(values[7:11])  # 提取 action，这里假设 action 是一个由四个元素组成的元组
        reward = values[-1]  # 提取 reward

        # 更新字典中对应 action 的累计 reward 和计数器
        if action in action_rewards:
            action_rewards[action]['total_reward'] += reward
            action_rewards[action]['count'] += 1
        else:
            action_rewards[action] = {'total_reward': reward, 'count': 1}

# 计算每个 action 的平均 reward
average_rewards = {}
for action, reward_info in action_rewards.items():
    average_reward = reward_info['total_reward'] / reward_info['count']
    average_rewards[action] = average_reward

# 提取每个动作的平均奖励值
average_rewards_values = list(average_rewards.values())

# 划分平均奖励区间
num_bins = 10
bins = np.linspace(min(average_rewards_values), max(average_rewards_values), num_bins + 1)

# 统计每个区间内动作的数量
action_counts = {i: 0 for i in range(num_bins)}
for reward in average_rewards_values:
    for i in range(num_bins):
        if bins[i] <= reward < bins[i+1]:
            action_counts[i] += 1
            break

# 绘制柱状图
plt.bar(range(num_bins), action_counts.values(), align='center', alpha=0.5)
plt.xticks(range(num_bins), [f"{bins[i]:.2f}-{bins[i+1]:.2f}" for i in range(num_bins)])
plt.xlabel('Average Reward Range')
plt.ylabel('Number of Actions')
plt.title('Number of Actions in Each Average Reward Range')
plt.grid(True)
plt.show()


In [ ]:
a = torch.rand(4,4)
torch.argmax(a, 1)

In [1]:
import random
import torch
import numpy as np
from torch.utils.data import DataLoader
import torch
from torch.utils.data import Dataset, SubsetRandomSampler
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim

class CustomDataset(Dataset):
    def __init__(self, tensor_paths, device):
        self.tensor_paths = tensor_paths
        self.device = device
    
    def __len__(self):
        return len(self.tensor_paths)
    
    def __getitem__(self, idx):
        tensor_path = self.tensor_paths[idx]
        q_value_map, q2_output, action, reward = load_tensor(tensor_path, self.device)
        return q_value_map, q2_output, action, reward

class CustomModel(nn.Module):
    def __init__(self, input1_channels, input2_size, input3_size, hidden_size, output_size):
        super(CustomModel, self).__init__()

        # Input1的处理
        self.conv1 = nn.Conv2d(input1_channels, 4, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(4, 8, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.maxpooling = nn.MaxPool2d(kernel_size=4)
        self.fc1 = nn.Linear(16384, 128)
        self.fc2 = nn.Linear(128, hidden_size)

        # Input2的处理
        self.fc3 = nn.Linear(input2_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        
        # Input3的处理
        self.fc5 = nn.Linear(input3_size, hidden_size)
        self.fc6 = nn.Linear(hidden_size, hidden_size)

        # 输出层
        self.output_layer = nn.Linear(hidden_size * 3, output_size)

    def forward(self, input1, input2, input3):
        # Input1的处理
        print(input1.shape)
        x1 = torch.relu(self.conv1(input1))
        x1 = torch.relu(self.conv2(x1))
        x1 = torch.relu(self.conv3(x1))
        x1 = self.maxpooling(x1)
        x1 = torch.flatten(x1, 1)
        print(x1.shape)
        x1 = torch.relu(self.fc1(x1))
        x1 = torch.relu(self.fc2(x1))

        # Input2的处理
        x2 = torch.relu(self.fc3(input2))
        x2 = torch.relu(self.fc4(x2))

        # Input3的处理
        x3 = torch.relu(self.fc5(input3))
        x3 = torch.relu(self.fc6(x3))

        # 将三个输入连接起来
        x = torch.cat((x1, x2, x3), dim=1)

        # 输出层
        output = self.output_layer(x)

        return output
    
def generate_path_list():
    path_list = [f"../dataset/{i}_{j}.pt" for i in range(86) for j in range(100)]
    return path_list

def load_tensor(tensor_path, device):
    tensor = torch.load(tensor_path, map_location=device).detach()
    q_value_map = tensor[:128*128].reshape(1,128,128)
    q2_output = tensor[128*128:-4]
    action = tensor[-4:-1]
    reward = tensor[-1:]
    return q_value_map, q2_output, action, reward

def set_random_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    random.seed(seed)
    
set_random_seed(42)

if torch.cuda.is_available():
    device_name = torch.device('cuda')
    print("found CUDA device")
else:
    # raise ValueError("cannot run on cpu device")
    device_name = torch.device('cpu')
    print("Warning: no CUDA device found, running on CPU")
    
set_random_seed(42)
data_path_list = generate_path_list()
data_num = len(data_path_list)
random.shuffle(data_path_list)


train_size = int(0.8 * len(data_path_list))
eval_size = int(0.1 * len(data_path_list))
test_size = len(data_path_list) - train_size - eval_size

train_dataset = CustomDataset(data_path_list[:train_size], device_name)
eval_dataset = CustomDataset(data_path_list[train_size:train_size+eval_size], device_name)
test_dataset = CustomDataset(data_path_list[train_size+eval_size:], device_name)

input1_channels = 1
input2_size = 16
input3_size = 3
hidden_size = 64
output_size = 1

batch_size = 1
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

MyModel = CustomModel(input1_channels, input2_size, input3_size, hidden_size, output_size)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(MyModel.parameters(), lr=0.001)

for a,b,c,d in train_loader:
    print(a.shape,b.shape,c.shape,d.shape)
    break

len(train_dataset), len(eval_dataset)



torch.Size([1, 1, 128, 128]) torch.Size([1, 16]) torch.Size([1, 3]) torch.Size([1, 1])


(6880, 860)

In [2]:
num_epochs = 25
log_train_loss = []
log_eval_loss = []
eval_interval = 5
for e in range(1):
    
    MyModel.train()
    train_loss = 0.0
    train_bar = tqdm(train_loader, desc=f'Epoch {e+1}/{num_epochs}, Training')
    for idx, data in enumerate(train_loader):
        q_value_map, q2_output, action, reward = data
        output = MyModel(q_value_map, q2_output, action)
        loss = criterion(output, reward)
        log_train_loss.append(loss.detach().item())
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        break
    
    avg_train_loss = train_loss / len(train_loader.dataset)
    
    MyModel.eval()
    eval_loss = 0.0
    eval_bar = tqdm(eval_loader, desc=f'Epoch {e+1}/{num_epochs}, Evaluating')
    with torch.no_grad():
        for idx, data in enumerate(eval_loader):
            q_value_map, q2_output, action, reward = data
            output = MyModel(q_value_map, q2_output, action)
            loss = criterion(output, reward)
            log_eval_loss.append(loss.detach().item())
            eval_loss += loss.item()
            break

    avg_eval_loss = eval_loss / len(eval_loader.dataset)
    
    print(f'Epoch {e+1}/{num_epochs}, Average Train Loss: {avg_train_loss:.4f}, Average Eval Loss: {avg_eval_loss:.4f}')
            
            
        
        
        
  

Epoch 1/25, Training:   0%|          | 0/6880 [00:00<?, ?it/s][W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


torch.Size([1, 1, 128, 128])
torch.Size([1, 16384])


torch.Size([1, 1, 128, 128])
torch.Size([1, 16384])
Epoch 1/25, Average Train Loss: 0.0001, Average Eval Loss: 0.0008


In [ ]:
log_train_loss

In [3]:
for i in range(10):
    k = i*0.5
    
print(k)

4.5


In [17]:
import torch
import torch.nn as nn
criterion = nn.BCEWithLogitsLoss()

input = torch.tensor([-8.], requires_grad=True)
target = torch.tensor([1.])

criterion(input, target)


tensor(8.0003, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [13]:
sig_input = torch.sigmoid(input)
sig_input

tensor([0.], grad_fn=<SigmoidBackward0>)

In [14]:
_criterion = nn.BCELoss()
_criterion(sig_input, target)

tensor(100., grad_fn=<BinaryCrossEntropyBackward0>)

In [19]:
with open("action.txt", 'r') as file:
    line_cnt = 0
    sr = 1
    data = []
    data_dict = {}
    for line in file:
        numbers = re.findall(r'[-+]?\d*\.\d+(?:[eE][-+]?\d+)?|\d+', line)
        numeric = [float(num) if '.' in num else int(num) for num in numbers]
        line_cnt += 1
        if line_cnt == 100:
            line_cnt = 0
            reward = numeric[-2:-1]
            print(reward)

[False]